# DAX expansion Liquidity - Python

### Overview
This notebook is the basis of this blog post : https://ganymde.cloud/dax-expansion.html

In this sample, we will get an insight on how an index impacts a stock liquidity. We will look back on how DAX index expansion from 30 to 40 stocks impacted liquidity for these new stocks leveraging several APIs we provide.

Following the announcement of Deutsche Böerse in 2020, the index became larger since <i>20 September 2021</i> by incorporating ten of the largest companies from Germany's MDAX listing 👇
* *Airbus SE, Brenntag SE, HelloFresh SE, Porsche SE, Puma SE, Qiagen NV, Sartorius AG (Pref. shares), Siemens Healthineers AG, Symrise AG and Zalando SE*

In order to understand the **index expansion impact on liquidity** for DAX components, we suggest to request analytical parameters suchs as `traded volumes`, `trades counts`. Each **trade** is pre-mapped, normalized and stored as a *tick trade* in our data store, readily available for on-the-fly analytics.

This samples enables to retrieve **on-demand** tick trades data points by calling a dedicated service.

### Inputs/outputs
Tick trades sample requires instruments' identifiers, date time intervals as per inputs. It returns the tick count sampled using the input time granularity.

We will request **tick trades data for the DAX components** before and after the index expansion, on *20 September 2021*. 
Then, we will compute **liquidity movements** for each new component in order to understand the `integration into index impact on liquidity`.

### Services used
This sample uses *gRPC requests* in order to retrieve ticks from the dedicated hosted service. The queried endpoint in this script are:
* TickTradesService: to directly retrieve tick trades objects from the server.

### Modules required
1. Systemathics packages:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* as per display package
    
***

# Run DAX expansion - Liquidity

### Step 1: Import required packages

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib --quiet

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.tick.v1.tick_trades_pb2 as tick_trades
import systemathics.apis.services.tick.v1.tick_trades_pb2_grpc as tick_trades_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Prepare request parameters

#### 3.1 Instruments' selection
The following code snippet enables to set the **DAX components**, before and after index expansion. 
As we are using *ICE data services* as market data source, tickers are simply the `ICE symbols`.

In [ ]:
old_components = ['E:ADSD', 'E:ALVD', 'E:BASD', 'E:BAYND', 'E:BEID', 'E:BMWD', 'E:CBKD', 'E:COND', 'E:DAID', 'E:DBKD', 'E:DB1D', 'E:LHAD', 'E:DPWD', 'E:DTED', 'E:EOAND', 'E:FRED', 'E:FMED', 'E:HEID', 'E:HEND', 'E:IFXD', 'E:LIND', 'E:MRKD', 'E:MUV2D', 'E:PSMD', 'E:RWED', 'E:SAPD', 'E:SIED', 'E:TKAD', 'E:VOWD', 'E:VNAD']
new_components = ['E:PUMD', 'E:ZALD', 'E:SHLD', 'E:SY1D', 'E:HFGD', 'E:SRTD', 'E:PAH3D', 'E:BNRD', 'E:QIAD', 'E:AIRP']

Each ticker is available in a specific ICE data location called `ICE data source` with a **source_Id**.

In [ ]:
source = '840' # ICE data location for DAX tickers after source_Id migration  occured on "date to be added"

#### 3.2 Trades request parameters
The following code snippet enables to set the required parameters to call **TickTradesService**.

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 6, day = 1), 
    end_date = date.Date(year = 2021, month = 12, day = 2)
)

# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

### Step 4: Retrieve data
The following code snippet creates a method to handle request creation to **TickTradesService** for a given instrument.

In [ ]:
# define method to handle tick trades request creation for each instrument
def get_trades_request(instrument_id):
    request = tick_trades.TickTradesRequest(
        identifiers = [instrument_id],
        constraints = constraints
    )
    return request

The following code snippet creates a method that calls **TickTradesService** and returns an array of two *pandas dataframes*:
* 1st dataframe with trade sizes 
* 2nd dataframe with trades counts

In [ ]:
# define method to handle tick trades data for a given list of instruments (old/new DAX components)
def get_trades_data(tickers):
    
    df_sizes = pd.DataFrame({'Date': []})
    df_sizes = df_sizes.set_index('Date')

    df_counts = pd.DataFrame({'Date': []})
    df_counts = df_counts.set_index('Date')
    
    # iterate all instrument identifiers: exhange/ticker pairs
    for i in range(len(tickers)):
        ticker = tickers[i]
        try:
            # open a gRPC channel
            with channel_helpers.get_grpc_channel() as channel:

                # instantiate the tick trades service
                instrument_id = identifier.Identifier(exchange = source, ticker = ticker)
                request = get_trades_request(instrument_id)
                service = tick_trades_service.TickTradesServiceStub(channel)

                # process the tick trades request
                sizes, counts = {}, {}
                for trade in service.TickTrades(request=request, metadata=[('authorization', token)]):
                    if trade.trade.price>0:
                        date = datetime.fromtimestamp(trade.trade.time_stamp.seconds).date()
                        size = trade.trade.size
                        if date in sizes:
                            sizes[date]+=size
                            counts[date] += 1
                        else:
                            sizes[date] = size
                            counts[date] = 1

                tmp_df_sizes = pd.DataFrame(data ={'Date': sizes.keys(), f'{ticker}': sizes.values()})
                tmp_df_sizes = tmp_df_sizes.set_index('Date')
                if (df_sizes.size == 0):
                    df_sizes = tmp_df_sizes
                else:
                    df_sizes = pd.merge(df_sizes, tmp_df_sizes, on="Date")

                tmp_df_counts = pd.DataFrame(data ={'Date': counts.keys(), f'{ticker}': counts.values()})
                tmp_df_counts = tmp_df_counts.set_index('Date')
                if (df_counts.size == 0):
                    df_counts = tmp_df_counts
                else:
                    df_counts = pd.merge(df_counts, tmp_df_counts, on="Date")
        except grpc.RpcError as e:
            display(e.code().name)
            display(e.details())

    return [df_sizes, df_counts]

#### 4.1 Retrive trades data for DAX previous composition
The following code snippet calls the previous method in order to retrieve **daily trade sizes** and **daily trades counts** for `DAX30, previous composition`:

In [ ]:
# create an array to store 2 dataframes (trade sizes & trades counts) for DAX previous composition, total of 30 tickers
#old_trades_data = get_trades_data(old_components)

In [ ]:
# access first array element: daily traded sizes for DAX previous composition
#old_trades_data[0]

In [ ]:
# access second array element: daily trades counts for DAX previous composition
#old_trades_data[1]

#### 4.2 Retrive trades data for DAX new components
The following code snippet calls the previous method in order to retrieve **daily traded sizes** and **daily trades counts** for `DAX new components`:

In [ ]:
# create an array to store 2 dataframes (traded sizes & trades counts) for DAX new components, total of 10 tickers
new_trades_data = get_trades_data(new_components)

In [ ]:
# access first array element: daily traded sizes for DAX new components
new_sizes = new_trades_data[0]
new_sizes

In [ ]:
# access second array element: daily trades counts for DAX new components
new_trades = new_trades_data[1]

In [ ]:
new_trades

### Step 5: Liquidity analysis following DAX extension

In [ ]:
names = ['Puma SE', 'Zalando SE', 'Siemens Healthineers AG', 'Symrise AG', 'HelloFresh SE', 'Sartorius AG', ' Porsche Automobil Holding SE', 'Brenntag SE', 'QIAGEN NV', 'Airbus SE']

In [ ]:
# we start from June 1st until December 2nd , including September 20th, DAX expansion date
start_date = datetime(2021, 6, 1).date()
end_date = datetime(2021, 12, 2).date()
change_date = datetime(2021, 9, 20).date()
day_before_change = datetime(2021, 9, 19).date()

# compute traded sizes percentage change
size_ratios = (new_sizes[change_date:end_date].mean() - new_sizes[start_date:day_before_change].mean()) / new_sizes[start_date:day_before_change].mean()
liquidity_size_ratios = round(size_ratios*100, 2)
liquidity_size_changes = pd.DataFrame({'Name': names,'Ticker': liquidity_size_ratios.index, 'Traded size movement (%)': liquidity_size_ratios.values})

# compute trade count percentage change 
count_ratios = (new_trades[change_date:end_date].mean() - new_trades[start_date:day_before_change].mean()) / new_trades[start_date:day_before_change].mean()
liquidity_count_ratios = round(count_ratios*100, 2)
liquidity_count_changes = pd.DataFrame({'Name': names,'Ticker': liquidity_count_ratios.index, 'Trades count movement (%)': liquidity_count_ratios.values})

# Merge both data frame
liquidity = pd.merge(liquidity_count_changes, liquidity_size_changes, on=["Ticker", "Name"])
liquidity